**Mount Drive & Install Packages**

First the directory containing the data must be mounted to the Colab virtual machine

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Initialise paths for the training and testing dataset






In [ ]:
training_data_path = '/content/drive/My Drive/AD Project/Alzheimers-Data-Preprocessed-N4/train/'
valid_data_path = '/content/drive/My Drive/AD Project/Alzheimers-Data-Preprocessed-N4/test/'

Import the appropriate packages

In [ ]:
import tensorflow as tf
from tensorflow import keras

tf.__version__
keras.__version__

In [ ]:
keras.backend.backend()

Load the images from the directory in the Google Drive

In [ ]:
trainDataset = tf.keras.preprocessing.image_dataset_from_directory(training_data_path, labels="inferred",color_mode="grayscale",batch_size=32, label_mode="categorical", image_size=(256,256))

validDataset = tf.keras.preprocessing.image_dataset_from_directory(valid_data_path, labels="inferred",color_mode="grayscale",batch_size=32, label_mode="categorical",image_size=(256,256))

Introduce the datagenerator to ensure that not all data is loaded at once - this could overload the google drive disk allocation

In [ ]:

trainGenerator = tf.keras.preprocessing.image.ImageDataGenerator()

validGenerator = tf.keras.preprocessing.image.ImageDataGenerator()

flowTrainGenerator = trainGenerator.flow_from_directory(training_data_path, target_size=(256,256), color_mode='grayscale',shuffle=False,interpolation="nearest",batch_size=128, class_mode='categorical')

flowValidGenerator = validGenerator.flow_from_directory(valid_data_path, target_size=(256,256), color_mode='grayscale',shuffle=False,interpolation="nearest",batch_size=128, class_mode='categorical')

#trainDatasetGenerator = tf.keras.preprocessing.image.ImageDataGenerator.flow_from_directory(trainDataset, target_size=(256,256), color_mode='grayscale',shuffle=True,interpolation="nearest",batch_size=32)

#validDatasetGenerator = tf.keras.preprocessing.image.ImageDataGenerator.flow_from_directory(validDataset, target_size=(256,256), color_mode='grayscale',shuffle=True,interpolation="nearest",batch_size=32)

Once data samples are loaded, the model can be constructed

In [ ]:
import cv2 as cv

imageShape = cv.imread('/content/drive/My Drive/AD Project/Alzheimers-Data-Preprocessed/train/Final CN JPEG/xxx.jpg')

print(imageShape.shape)

print(imageShape.dtype)

**Implementation of AlexNet**






In [ ]:
#Use sequential to build AlexNet neural network

modelv2Alex = tf.keras.Sequential()

#Define the first input shape of the model - 256x256 pixels and 3 channels of colour - obtained above using openCV

inputShape = (256,256,1)

#First convolutional layer 

modelv2Alex.add(tf.keras.layers.Conv2D(96,(11,11), input_shape=inputShape, strides = (4,4), padding='valid'))

#Batch normalisation before activation function

modelv2Alex.add(tf.keras.layers.BatchNormalization())

#Add activation function

modelv2Alex.add(tf.keras.layers.Activation('relu'))

#Max pooling layer

modelv2Alex.add(tf.keras.layers.MaxPool2D(pool_size = (3), strides = (2,2),padding='valid'))

#Second convolutional layer

modelv2Alex.add(tf.keras.layers.Conv2D(256,(5,5), padding='same')) 

#Batch normalisation before activation function

modelv2Alex.add(tf.keras.layers.BatchNormalization()) 

#Add activation function

modelv2Alex.add(tf.keras.layers.Activation('relu'))

#Max pooling layer

modelv2Alex.add(tf.keras.layers.MaxPool2D(pool_size = (3), strides = (2,2),padding='valid'))

#Third convolutional layer

modelv2Alex.add(tf.keras.layers.Conv2D(256,(3,3), padding='same')) 

#Batch normalisation before activation function

modelv2Alex.add(tf.keras.layers.BatchNormalization()) 

#Add activation function

modelv2Alex.add(tf.keras.layers.Activation('relu'))

#Fourth convolutional layer

modelv2Alex.add(tf.keras.layers.Conv2D(256,(3,3), padding='same')) 

#Batch normalisation before activation function

modelv2Alex.add(tf.keras.layers.BatchNormalization()) 

#Add activation function

modelv2Alex.add(tf.keras.layers.Activation('relu'))

#Fifth convolutional layer

modelv2Alex.add(tf.keras.layers.Conv2D(256,(3,3), padding='same')) 

#Batch normalisation before activation function

modelv2Alex.add(tf.keras.layers.BatchNormalization()) 

#Add activation function

modelv2Alex.add(tf.keras.layers.Activation('relu'))

#Max pooling layer

modelv2Alex.add(tf.keras.layers.MaxPool2D(pool_size = (3), strides = (2,2)))

#Flatten 

modelv2Alex.add(tf.keras.layers.Flatten())

#First fully connected layer

modelv2Alex.add(tf.keras.layers.Dense(4096, activation = "relu"))

#Dropout layer 1

modelv2Alex.add(tf.keras.layers.Dropout(rate=0.5))

#Second fully connected layer

modelv2Alex.add(tf.keras.layers.Dense(4096, activation = "relu"))

#Dropout layer 1

modelv2Alex.add(tf.keras.layers.Dropout(rate=0.5))

#Third fully connected layer

modelv2Alex.add(tf.keras.layers.Dense(5, activation = "softmax"))

#Initialise optimisers to give fine control over parameters

SGDoptimiser = tf.keras.optimizers.SGD(momentum = 0.9, lr=0.01)

RMSPropOptimiser = tf.keras.optimizers.SGD(lr=0.01, momentum=0.9)

#compile the above model and specify the optimisation function for Stochastic Gradient Decent and categorical crossentropy loss function

modelv2Alex.compile(optimizer=SGDoptimiser, loss='categorical_crossentropy', metrics=['accuracy',tf.keras.metrics.Recall(),tf.keras.metrics.Precision(), tf.keras.metrics.TruePositives(thresholds=None, name=None, dtype=None), tf.keras.metrics.TrueNegatives(thresholds=None, name=None, dtype=None), tf.keras.metrics.FalsePositives(thresholds=None, name=None, dtype=None), tf.keras.metrics.FalseNegatives(thresholds=None, name=None, dtype=None)])

#Print message to indicate model compilation has been completed

print('Model AlexNet compilation complete')

#Print the model summary to gain insight

print(modelv2Alex.summary())

modelv2Alex.save('/content/drive/My Drive/AD Project/Model Saves/')



In [ ]:
#Fit the AlexNet model to data and assess performance

alexNetFitted = modelv2Alex.fit(flowTrainGenerator, epochs=400, steps_per_epoch=9, validation_data=flowValidGenerator, validation_steps=2, verbose=1)

**Visualisations of Performance**

In [ ]:
import matplotlib.pyplot as plt

print(alexNetFitted.history)

#Plot the accuracy change by number of epochs

plt.plot(alexNetFitted.history["accuracy"])
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.title("Accuracy by Epoch")
plt.show()

#Plot the change in loss by number of epochs

plt.plot(alexNetFitted.history["loss"])
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.title("Loss by Epoch")
plt.show()

#Plot the validation accuracy change by number of epochs

plt.plot(alexNetFitted.history["val_accuracy"])
plt.ylabel("Validation Accuracy")
plt.xlabel("Epoch")
plt.title("Validation Accuracy by Epoch")
plt.show()

#Plot the change in validaiton loss by number of epochs

plt.plot(alexNetFitted.history["val_loss"])
plt.ylabel("Validation Loss")
plt.xlabel("Epoch")
plt.title("Validation Loss by Epoch")
plt.show()


**Obtain Metrics**

Calculation of F-1 Score using precision and recall metrics and plot confusion matrix.

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, ConfusionMatrixDisplay

F1_Score = (2*(((alexNetFitted.history['val_precision_1'][-1])*(alexNetFitted.history['val_recall_1'][-1]))/((alexNetFitted.history['val_precision_1'][-1])+(alexNetFitted.history['val_recall_1'][-1]))))

print("Accuracy is:")

print(alexNetFitted.history['accuracy'][-1])

print("F-1 Score is:")

print(F1_Score)

print("Precision is:")

print(alexNetFitted.history['val_precision_1'][-1])

print("Recall is:")

print(alexNetFitted.history['val_recall_1'][-1])

print("Confusion Matrix")

classPrediction = modelv2Alex.predict(flowValidGenerator, verbose=1)

classPrediction = np.argmax(classPrediction, axis=1)

#Use skikit learn maxtrix generator to make confusion matrix with numpy array

confusionMatrix = confusion_matrix(flowValidGenerator.classes,classPrediction)

displayMatrix = ConfusionMatrixDisplay(confusionMatrix, display_labels=flowTrainGenerator.classes)

displayMatrix.plot()